# Workflow registry authorization

In [1]:
lifemonitor_root = "/home/simleo/git/life_monitor"
%cd -q {lifemonitor_root}

In [2]:
import requests

In [3]:
# Get info on the "seek" registry
!docker-compose exec lm /bin/bash -c "flask registry show seek"



****************************************************************************************************
Workflow Registry 'seek' (uuid: 8ee6f93b-ac01-4183-b90e-14fec7ac9d22, type: seek_registry) registered!
****************************************************************************************************


OAuth2 settings to connect to LifeMonitor:
----------------------------------------------------------------------------------------------------
REGISTRY NAME: seek
REGISTRY API URL: https://seek:3000
REGISTRY CLIENT ID: fOeRRL3Z7tI1i5iHRczz4B0X
REGISTRY CLIENT SECRET: WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE
REGISTRY CLIENT ALLOWED SCOPES: registry.info registry.user registry.workflow.read registry.workflow.write registry.user.workflow.read registry.user.workflow.write workflow.read workflow.write testingService.read testingService.write user.profile user.workflow.read
REGISTRY CLIENT ALLOWED FLOWS: ['client_credentials', 'authorization_code', 'refresh_token']
REGISTRY CLIEN

In [4]:
# Copy registry credentials from the above dump
CLIENT_ID = "fOeRRL3Z7tI1i5iHRczz4B0X"
CLIENT_SECRET = "WA1rPDSudxDcQsZpqZosQXGwMx57hghbdMCaE7FBe9OmQLqE"

In [5]:
s = requests.session() # not mandatory, just to share some settings among requests
s.verify = False  # disable SSL verification so we can use self-signed certificates
s.headers.update({})

In [6]:
lm_base_url = "https://localhost:8443"
lm_token_url = f"{lm_base_url}/oauth2/token"

In [7]:
# Get an authorization token from LifeMonitor
token_response = s.post(
    lm_token_url, 
    data={
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "grant_type": "client_credentials",
        "scope": "registry.info"
    }, allow_redirects=True, verify=False)
assert token_response.status_code == 200, "OAuth2 Error"
token = token_response.json()
token

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'access_token': 'oOHsnazv09vyWxFXRLpZpANujruDCe71eaFAoQaUao',
 'expires_in': 864000,
 'scope': 'registry.info',
 'token_type': 'Bearer'}

In [8]:
# Update headers with the OAuth2 token
s.headers.update({'Authorization': f"Bearer {token['access_token']}"})

In [9]:
# Get registry info
response = s.get(f"{lm_base_url}/registries/current")
assert response.status_code == 200, f"Unexpected error {response.status_code}: {response.content}"
current_registry = response.json()
current_registry

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:860: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'meta': {'api_version': '0.2.0-beta2',
  'base_url': 'https://172.19.0.4:8000',
  'created': '2021-04-27T13:31:30.601893',
  'modified': '2021-04-27T13:31:30.601906',
  'resource': '/registries/current'},
 'name': 'seek',
 'type': 'seek',
 'uri': 'https://seek:3000',
 'uuid': '8ee6f93b-ac01-4183-b90e-14fec7ac9d22'}